In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import sklearn
import numpy as np
from sklearn.metrics import (mean_squared_error, mean_absolute_error,r2_score)
import pickle

In [2]:
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
tf.__version__

'2.9.1'

In [4]:
df = pd.read_csv(r"Dataset.txt",na_values='?')

In [5]:
df = pd.read_csv(r"Dataset.txt",na_values='?')

In [6]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


# EDA

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.1+ KB


In [8]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
car name        0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
car name        0
dtype: int64

In [11]:
df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [12]:
df['car name'].unique()

array(['chevrolet chevelle malibu ', 'buick skylark 320',
       'plymouth satellite', 'amc rebel sst', 'ford torino',
       'ford galaxie 500', 'chevrolet impala', 'plymouth fury iii',
       'pontiac catalina', 'amc ambassador dpl', 'dodge challenger se',
       "plymouth 'cuda 340", 'chevrolet monte carlo',
       'buick estate wagon (sw)', 'toyota corona mark ii',
       'plymouth duster', 'amc hornet', 'ford maverick', 'datsun pl510',
       'volkswagen 1131 deluxe sedan', 'peugeot 504', 'audi 100 ls',
       'saab 99e', 'bmw 2002', 'amc gremlin', 'ford f250', 'chevy c20',
       'dodge d200', 'hi 1200d', 'chevrolet vega 2300', 'toyota corona',
       'plymouth satellite custom', 'chevrolet chevelle malibu',
       'ford torino 500', 'amc matador', 'pontiac catalina brougham',
       'dodge monaco (sw)', 'ford country squire (sw)',
       'pontiac safari (sw)', 'amc hornet sportabout (sw)',
       'chevrolet vega (sw)', 'pontiac firebird', 'ford mustang',
       'mercury capri 20

# Drop Car Name 

In [13]:
df.drop(['car name'],axis=1,inplace=True)

# Splitting in Train and Test

In [14]:
X=df.drop("mpg",axis=1)

In [15]:
Y=df['mpg']

In [16]:
X

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,8,307.0,130.0,3504,12.0,70,1
1,8,350.0,165.0,3693,11.5,70,1
2,8,318.0,150.0,3436,11.0,70,1
3,8,304.0,150.0,3433,12.0,70,1
4,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,1
394,4,97.0,52.0,2130,24.6,82,2
395,4,135.0,84.0,2295,11.6,82,1
396,4,120.0,79.0,2625,18.6,82,1


In [17]:
Y

0      18.0
1      15.0
2      18.0
3      16.0
4      17.0
       ... 
393    27.0
394    44.0
395    32.0
396    28.0
397    31.0
Name: mpg, Length: 392, dtype: float64

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=101)

In [20]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
274,5,131.0,103.0,2830,15.9,78,2
157,8,350.0,145.0,4440,14.0,75,1
173,4,119.0,97.0,2545,17.0,75,3
357,4,119.0,100.0,2615,14.8,81,3
263,6,231.0,165.0,3445,13.4,78,1


In [21]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [22]:
X_train_std=sc.fit_transform(X_train)
X_test_std=sc.transform(X_test)

In [23]:
X_train_std

array([[-0.24893345, -0.57765698, -0.02197412, ...,  0.11961844,
         0.54658592,  0.49337407],
       [ 1.52188861,  1.52787598,  1.06400983, ..., -0.55787761,
        -0.26938257, -0.74203461],
       [-0.83920747, -0.69302865, -0.17711468, ...,  0.51185299,
        -0.26938257,  1.72878276],
       ...,
       [ 0.34134057, -0.17385614,  0.15902321, ..., -1.05708522,
         1.36255441, -0.74203461],
       [ 1.52188861,  1.43173292,  1.45186124, ..., -2.6973388 ,
        -1.62933005, -0.74203461],
       [-0.83920747, -0.79878601, -0.74596341, ...,  0.44053762,
         1.36255441,  1.72878276]])

In [24]:
X_test_std

array([[ 1.52188861e+00,  2.00859127e+00,  1.83971265e+00,
         1.78911930e+00, -1.44931977e+00, -1.35734055e+00,
        -7.42034609e-01],
       [ 1.52188861e+00,  1.09523222e+00,  6.76158422e-01,
         1.04842530e+00, -5.86699954e-02,  8.18575415e-01,
        -7.42034609e-01],
       [ 3.41340567e-01,  3.83773597e-01,  1.59023209e-01,
         9.76305932e-02, -2.01300742e-01, -2.69382568e-01,
        -7.42034609e-01],
       [-8.39207472e-01, -1.01991504e+00, -8.75247218e-01,
        -1.20095793e+00,  4.40537616e-01,  2.74596424e-01,
         1.72878276e+00],
       [ 3.41340567e-01,  3.26087762e-01, -2.28828201e-01,
         9.83139770e-01,  1.22500672e+00, -2.69382568e-01,
        -7.42034609e-01],
       [ 3.41340567e-01,  5.66445406e-01, -6.68393133e-01,
         7.32680740e-01,  1.93816045e+00,  2.60692807e-03,
        -7.42034609e-01],
       [-8.39207472e-01, -1.14490101e+00, -9.01103979e-01,
        -1.59504513e+00,  8.68429855e-01, -1.35734055e+00,
         1.7287827

# saving SC transform

In [25]:
pickle.dump(sc, open(r'D:\Python37\Projects\Fuel Efficiency Prediction using ANN\scaler.pkl', 'wb'))

In [26]:
Y_train

274    20.3
157    15.0
173    24.0
357    32.9
263    17.7
       ... 
64     15.0
328    30.0
341    23.5
11     14.0
356    32.4
Name: mpg, Length: 313, dtype: float64

In [27]:
Y_test

39     14.0
285    17.0
164    21.0
220    33.5
163    18.0
       ... 
17     21.0
82     23.0
58     25.0
344    39.0
283    20.2
Name: mpg, Length: 79, dtype: float64

In [28]:
X_train.keys()

Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'model_year', 'origin'],
      dtype='object')

In [29]:
len(X_train.keys())

7

In [30]:
# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(units = 64,activation='relu',input_shape=[len(X_train.keys())]))
# Adding the second hidden layer
model.add(Dense(units = 64,activation='relu'))
model.add(Dense(units = 64,activation='relu'))
model.add(Dense(units = 64,activation='relu'))
model.add(Dense(units=1))
# Adding the output layer

# Compiling the ANN
model.compile(loss='mean_squared_error',
                optimizer='Adam',
                metrics=['mean_absolute_error', 'mean_squared_error'])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,057
Trainable params: 13,057
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Fitting the ANN to the Training set
model_history=model.fit(X_train_std, Y_train, batch_size = 100,epochs = 1000)

Epoch 1/1000
4/4 [==============================] - 0s 997us/step - loss: 611.3524 - mean_absolute_error: 23.4292 - mean_squared_error: 611.3524
Epoch 2/1000
4/4 [==============================] - 0s 997us/step - loss: 594.1643 - mean_absolute_error: 23.0456 - mean_squared_error: 594.1643
Epoch 3/1000
4/4 [==============================] - 0s 997us/step - loss: 571.6149 - mean_absolute_error: 22.5428 - mean_squared_error: 571.6149
Epoch 4/1000
4/4 [==============================] - 0s 1ms/step - loss: 539.9848 - mean_absolute_error: 21.8196 - mean_squared_error: 539.9848
Epoch 5/1000
4/4 [==============================] - 0s 997us/step - loss: 492.8273 - mean_absolute_error: 20.7142 - mean_squared_error: 492.8273
Epoch 6/1000
4/4 [==============================] - 0s 997us/step - loss: 427.6135 - mean_absolute_error: 19.0442 - mean_squared_error: 427.6135
Epoch 7/1000
4/4 [==============================] - 0s 998us/step - loss: 340.1985 - mean_absolute_error: 16.6206 - mean_squared_err

4/4 [==============================] - 0s 1ms/step - loss: 8.8785 - mean_absolute_error: 2.1205 - mean_squared_error: 8.8785
Epoch 59/1000
4/4 [==============================] - 0s 1ms/step - loss: 8.7891 - mean_absolute_error: 2.1253 - mean_squared_error: 8.7891
Epoch 60/1000
4/4 [==============================] - 0s 1ms/step - loss: 8.8706 - mean_absolute_error: 2.1588 - mean_squared_error: 8.8706
Epoch 61/1000
4/4 [==============================] - 0s 1ms/step - loss: 8.8098 - mean_absolute_error: 2.1527 - mean_squared_error: 8.8098
Epoch 62/1000
4/4 [==============================] - 0s 1ms/step - loss: 8.5020 - mean_absolute_error: 2.1057 - mean_squared_error: 8.5020
Epoch 63/1000
4/4 [==============================] - 0s 997us/step - loss: 8.4169 - mean_absolute_error: 2.0518 - mean_squared_error: 8.4169
Epoch 64/1000
4/4 [==============================] - 0s 998us/step - loss: 8.7017 - mean_absolute_error: 2.0914 - mean_squared_error: 8.7017
Epoch 65/1000
4/4 [==================

4/4 [==============================] - 0s 997us/step - loss: 6.5592 - mean_absolute_error: 1.8041 - mean_squared_error: 6.5592
Epoch 117/1000
4/4 [==============================] - 0s 1ms/step - loss: 6.5502 - mean_absolute_error: 1.7990 - mean_squared_error: 6.5502
Epoch 118/1000
4/4 [==============================] - 0s 1ms/step - loss: 6.6444 - mean_absolute_error: 1.8064 - mean_squared_error: 6.6444
Epoch 119/1000
4/4 [==============================] - 0s 1ms/step - loss: 6.4588 - mean_absolute_error: 1.7959 - mean_squared_error: 6.4588
Epoch 120/1000
4/4 [==============================] - 0s 997us/step - loss: 6.6277 - mean_absolute_error: 1.8344 - mean_squared_error: 6.6277
Epoch 121/1000
4/4 [==============================] - 0s 1ms/step - loss: 6.4255 - mean_absolute_error: 1.7882 - mean_squared_error: 6.4255
Epoch 122/1000
4/4 [==============================] - 0s 1ms/step - loss: 6.6062 - mean_absolute_error: 1.8284 - mean_squared_error: 6.6062
Epoch 123/1000
4/4 [===========

4/4 [==============================] - 0s 997us/step - loss: 5.7827 - mean_absolute_error: 1.7132 - mean_squared_error: 5.7827
Epoch 175/1000
4/4 [==============================] - 0s 2ms/step - loss: 5.4514 - mean_absolute_error: 1.6472 - mean_squared_error: 5.4514
Epoch 176/1000
4/4 [==============================] - 0s 1ms/step - loss: 5.3976 - mean_absolute_error: 1.6379 - mean_squared_error: 5.3976
Epoch 177/1000
4/4 [==============================] - 0s 997us/step - loss: 5.4160 - mean_absolute_error: 1.6339 - mean_squared_error: 5.4160
Epoch 178/1000
4/4 [==============================] - 0s 997us/step - loss: 5.3565 - mean_absolute_error: 1.6185 - mean_squared_error: 5.3565
Epoch 179/1000
4/4 [==============================] - 0s 997us/step - loss: 5.3879 - mean_absolute_error: 1.6302 - mean_squared_error: 5.3879
Epoch 180/1000
4/4 [==============================] - 0s 997us/step - loss: 5.4600 - mean_absolute_error: 1.6575 - mean_squared_error: 5.4600
Epoch 181/1000
4/4 [=====

4/4 [==============================] - 0s 1ms/step - loss: 5.4500 - mean_absolute_error: 1.6352 - mean_squared_error: 5.4500
Epoch 233/1000
4/4 [==============================] - 0s 998us/step - loss: 4.8813 - mean_absolute_error: 1.5801 - mean_squared_error: 4.8813
Epoch 234/1000
4/4 [==============================] - 0s 1ms/step - loss: 5.1857 - mean_absolute_error: 1.6326 - mean_squared_error: 5.1857
Epoch 235/1000
4/4 [==============================] - 0s 997us/step - loss: 4.7864 - mean_absolute_error: 1.5630 - mean_squared_error: 4.7864
Epoch 236/1000
4/4 [==============================] - 0s 997us/step - loss: 4.6886 - mean_absolute_error: 1.5478 - mean_squared_error: 4.6886
Epoch 237/1000
4/4 [==============================] - 0s 1ms/step - loss: 5.0708 - mean_absolute_error: 1.6372 - mean_squared_error: 5.0708
Epoch 238/1000
4/4 [==============================] - 0s 997us/step - loss: 4.8941 - mean_absolute_error: 1.5828 - mean_squared_error: 4.8941
Epoch 239/1000
4/4 [=======

...

4/4 [==============================] - 0s 997us/step - loss: 1.8072 - mean_absolute_error: 0.8517 - mean_squared_error: 1.8072
Epoch 754/1000
4/4 [==============================] - 0s 997us/step - loss: 1.7565 - mean_absolute_error: 0.8301 - mean_squared_error: 1.7565
Epoch 755/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.8254 - mean_absolute_error: 0.8433 - mean_squared_error: 1.8254
Epoch 756/1000
4/4 [==============================] - 0s 998us/step - loss: 2.0946 - mean_absolute_error: 0.9378 - mean_squared_error: 2.0946
Epoch 757/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.8166 - mean_absolute_error: 0.8548 - mean_squared_error: 1.8166
Epoch 758/1000
4/4 [==============================] - 0s 997us/step - loss: 1.9003 - mean_absolute_error: 0.8777 - mean_squared_error: 1.9003
Epoch 759/1000
4/4 [==============================] - 0s 997us/step - loss: 1.8015 - mean_absolute_error: 0.8504 - mean_squared_error: 1.8015
Epoch 760/1000
4/4 [=====

4/4 [==============================] - 0s 1ms/step - loss: 1.7327 - mean_absolute_error: 0.8665 - mean_squared_error: 1.7327
Epoch 812/1000
4/4 [==============================] - 0s 997us/step - loss: 1.6728 - mean_absolute_error: 0.8282 - mean_squared_error: 1.6728
Epoch 813/1000
4/4 [==============================] - 0s 997us/step - loss: 1.6484 - mean_absolute_error: 0.8095 - mean_squared_error: 1.6484
Epoch 814/1000
4/4 [==============================] - 0s 997us/step - loss: 1.6745 - mean_absolute_error: 0.8258 - mean_squared_error: 1.6745
Epoch 815/1000
4/4 [==============================] - 0s 997us/step - loss: 1.6917 - mean_absolute_error: 0.8132 - mean_squared_error: 1.6917
Epoch 816/1000
4/4 [==============================] - 0s 997us/step - loss: 1.7123 - mean_absolute_error: 0.8246 - mean_squared_error: 1.7123
Epoch 817/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.8066 - mean_absolute_error: 0.8137 - mean_squared_error: 1.8066
Epoch 818/1000
4/4 [=====

4/4 [==============================] - 0s 997us/step - loss: 1.7262 - mean_absolute_error: 0.8404 - mean_squared_error: 1.7262
Epoch 870/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.6584 - mean_absolute_error: 0.8298 - mean_squared_error: 1.6584
Epoch 871/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.5965 - mean_absolute_error: 0.7752 - mean_squared_error: 1.5965
Epoch 872/1000
4/4 [==============================] - 0s 997us/step - loss: 1.4987 - mean_absolute_error: 0.7466 - mean_squared_error: 1.4987
Epoch 873/1000
4/4 [==============================] - 0s 998us/step - loss: 1.4787 - mean_absolute_error: 0.7309 - mean_squared_error: 1.4787
Epoch 874/1000
4/4 [==============================] - 0s 997us/step - loss: 1.5593 - mean_absolute_error: 0.7827 - mean_squared_error: 1.5593
Epoch 875/1000
4/4 [==============================] - 0s 997us/step - loss: 1.8365 - mean_absolute_error: 0.8490 - mean_squared_error: 1.8365
Epoch 876/1000
4/4 [=====

Epoch 927/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.3741 - mean_absolute_error: 0.6714 - mean_squared_error: 1.3741
Epoch 928/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.4216 - mean_absolute_error: 0.7207 - mean_squared_error: 1.4216
Epoch 929/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.4972 - mean_absolute_error: 0.7719 - mean_squared_error: 1.4972
Epoch 930/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.3888 - mean_absolute_error: 0.6873 - mean_squared_error: 1.3888
Epoch 931/1000
4/4 [==============================] - 0s 997us/step - loss: 1.4224 - mean_absolute_error: 0.7122 - mean_squared_error: 1.4224
Epoch 932/1000
4/4 [==============================] - 0s 997us/step - loss: 1.4059 - mean_absolute_error: 0.7048 - mean_squared_error: 1.4059
Epoch 933/1000
4/4 [==============================] - 0s 997us/step - loss: 1.3959 - mean_absolute_error: 0.6910 - mean_squared_error: 1.3959
Epoch 934/1000

4/4 [==============================] - 0s 997us/step - loss: 1.3316 - mean_absolute_error: 0.6607 - mean_squared_error: 1.3316
Epoch 986/1000
4/4 [==============================] - 0s 997us/step - loss: 1.2441 - mean_absolute_error: 0.6304 - mean_squared_error: 1.2441
Epoch 987/1000
4/4 [==============================] - 0s 997us/step - loss: 1.3635 - mean_absolute_error: 0.7088 - mean_squared_error: 1.3635
Epoch 988/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.2614 - mean_absolute_error: 0.6582 - mean_squared_error: 1.2614
Epoch 989/1000
4/4 [==============================] - 0s 997us/step - loss: 1.2636 - mean_absolute_error: 0.6495 - mean_squared_error: 1.2636
Epoch 990/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.3090 - mean_absolute_error: 0.6762 - mean_squared_error: 1.3090
Epoch 991/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.3057 - mean_absolute_error: 0.6680 - mean_squared_error: 1.3057
Epoch 992/1000
4/4 [=======

In [33]:
plt.scatter(Y_test,prediction)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

NameError: name 'prediction' is not defined

In [ ]:
model.save(r"C:\Users\shrey\OneDrive\Desktop\Fuel  Efficiency Prediction\Ne2w\Fuel-Efficiency-Prediction-using-ANN-main\models\model.h5")

In [34]:
X_test_std

array([[ 1.52188861e+00,  2.00859127e+00,  1.83971265e+00,
         1.78911930e+00, -1.44931977e+00, -1.35734055e+00,
        -7.42034609e-01],
       [ 1.52188861e+00,  1.09523222e+00,  6.76158422e-01,
         1.04842530e+00, -5.86699954e-02,  8.18575415e-01,
        -7.42034609e-01],
       [ 3.41340567e-01,  3.83773597e-01,  1.59023209e-01,
         9.76305932e-02, -2.01300742e-01, -2.69382568e-01,
        -7.42034609e-01],
       [-8.39207472e-01, -1.01991504e+00, -8.75247218e-01,
        -1.20095793e+00,  4.40537616e-01,  2.74596424e-01,
         1.72878276e+00],
       [ 3.41340567e-01,  3.26087762e-01, -2.28828201e-01,
         9.83139770e-01,  1.22500672e+00, -2.69382568e-01,
        -7.42034609e-01],
       [ 3.41340567e-01,  5.66445406e-01, -6.68393133e-01,
         7.32680740e-01,  1.93816045e+00,  2.60692807e-03,
        -7.42034609e-01],
       [-8.39207472e-01, -1.14490101e+00, -9.01103979e-01,
        -1.59504513e+00,  8.68429855e-01, -1.35734055e+00,
         1.7287827

In [35]:
prediction=model.predict(X_test_std)

3/3 [==============================] - 0s 0s/step


In [36]:
prediction

array([[13.6709795],
       [18.367046 ],
       [18.281628 ],
       [33.321877 ],
       [17.238213 ],
       [15.134082 ],
       [31.08131  ],
       [20.583652 ],
       [20.47931  ],
       [24.52002  ],
       [24.822208 ],
       [26.728394 ],
       [29.939629 ],
       [36.551598 ],
       [25.476818 ],
       [19.886272 ],
       [14.17015  ],
       [23.183554 ],
       [11.328617 ],
       [18.581835 ],
       [26.416653 ],
       [20.689457 ],
       [18.394276 ],
       [16.373672 ],
       [32.73161  ],
       [31.10731  ],
       [19.166336 ],
       [12.564051 ],
       [20.368994 ],
       [14.532991 ],
       [37.204037 ],
       [35.928688 ],
       [32.95294  ],
       [27.480986 ],
       [30.208252 ],
       [23.146616 ],
       [22.972435 ],
       [17.254307 ],
       [11.312656 ],
       [31.586506 ],
       [29.500103 ],
       [24.153765 ],
       [15.276749 ],
       [11.608097 ],
       [13.01877  ],
       [25.395512 ],
       [28.079012 ],
       [20.47

In [37]:
rmse=np.sqrt(mean_squared_error(Y_test,prediction))

print('RMSE: ',rmse)

print('R_square:', r2_score(Y_test,prediction))


RMSE:  3.284772887318204
R_square: 0.8020327711016563


# Lets predict for a single car

In [38]:
list1=[[8,307,130,3504,12,70,1]]
list1=sc.transform(list1)
list1

c:\python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.52188861,  1.11446083,  0.67615842,  0.64959007, -1.27103134,
        -1.62933005, -0.74203461]])

In [39]:
prediction2=model.predict(list1)

1/1 [==============================] - 0s 13ms/step


In [40]:
prediction2

array([[17.539635]], dtype=float32)

In [41]:
list2=[[8,307,130,3504,12,70,1]]
list2=sc.transform(list2)
list2

c:\python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.52188861,  1.11446083,  0.67615842,  0.64959007, -1.27103134,
        -1.62933005, -0.74203461]])

In [42]:
prediction3=model.predict(list2)
prediction3

1/1 [==============================] - 0s 13ms/step


array([[17.539635]], dtype=float32)

In [43]:
model.save("FuelEfficiency.model")

INFO:tensorflow:Assets written to: FuelEfficiency.model\assets
